### Data Access

In [0]:
App_id = 'b643d0ee-6423-42f7-b40c-11ae844a6466'
secret = '1d2bff7f-49af-46f6-bea0-21bd6406c239'
Dirid='c71668eb-b8c1-489a-acc6-2f62bbc48e24'



In [0]:
spark.conf.set("fs.azure.account.auth.type.nycstoragebishwajit.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.nycstoragebishwajit.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.nycstoragebishwajit.dfs.core.windows.net", "b643d0ee-6423-42f7-b40c-11ae844a6466")
spark.conf.set("fs.azure.account.oauth2.client.secret.nycstoragebishwajit.dfs.core.windows.net", "-1y8Q~rqByVjv3-C8Lw9s2Qhso01o_qD-IqsNc0.")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.nycstoragebishwajit.dfs.core.windows.net", "https://login.microsoftonline.com/c71668eb-b8c1-489a-acc6-2f62bbc48e24/oauth2/token")

In [0]:
dbutils.fs.ls("abfss://bronze@nycstoragebishwajit.dfs.core.windows.net/")


### **Data Reading **
**Importing Libraries**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


Reading CSV Data

**Trip Type Data**

In [0]:
df_trip_type = spark.read.format('csv') \
    .option("inferSchema", True) \
    .option("header", True) \
    .load("abfss://bronze@nycstoragebishwajit.dfs.core.windows.net/trip_type/")

In [0]:
dbutils.fs.ls("abfss://bronze@nycstoragebishwajit.dfs.core.windows.net/")


In [0]:
df_trip_type.display()

**Trip **Zone****

In [0]:
df_trip_zone = spark.read.format('csv') \
    .option("inferSchema", True) \
    .option("header", True)\
    .load("abfss://bronze@nycstoragebishwajit.dfs.core.windows.net/trip-zone")

df_trip_zone.display()

**Trip Data**

In [0]:
df_trip = spark.read.format('parquet') \
    .schema(myschema) \
    .option("header", True) \
    .option("recursiveFileLookup", True) \
    .load("abfss://bronze@nycstoragebishwajit.dfs.core.windows.net/trip2023data")

In [0]:
myschema = """
                VendorID BIGINT,
                lpep_pickup_datetime TIMESTAMP,
                lpep_dropoff_datetime TIMESTAMP,
                store_and_fwd_flag STRING,
                RatecodeID BIGINT,
                PULocationID BIGINT,
                DOLocationID BIGINT,
                passenger_count BIGINT,
                trip_distance DOUBLE,
                fare_amount DOUBLE,
                extra DOUBLE,
                mta_tax DOUBLE,
                tip_amount DOUBLE,
                tolls_amount DOUBLE,
                chail_fee DOUBLE,
                improvement_surcharge DOUBLE,
                total_amount DOUBLE,
                payment_type BIGINT,
                trip_type BIGINT,
                congestion_surcharge DOUBLE
           """

In [0]:
df_trip.display()

### **Data Transformation**

In [0]:
df_trip_type.display()

In [0]:
df_trip_type = df_trip_type.withColumnRenamed('description', 'trip_description')
df_trip_type.display()

In [0]:
df_trip_type.write.format('parquet')\
        .mode('append')\
        .option('path','abfss://silver@nycstoragebishwajit.dfs.core.windows.net/trip_type').save()

In [0]:
df_trip_zone.display()

In [0]:
df_trip_zone = df_trip_zone.withColumn('zone1',split(col('Zone'),'/')[0])\
                            .withColumn('zone2',split(col('Zone'),'/')[1])

df_trip_zone.display()

In [0]:
df_trip_zone.write.format('parquet')\
            .mode('append')\
            .option('path', 'abfss://silver@nycstoragebishwajit.dfs.core.windows.net/trip_zone').save()

In [0]:
df_trip.display()

In [0]:
df_trip = df_trip.withColumn('trip_date', to_date('lpep_pickup_datetime'))\
                .withColumn('trip_year', year('lpep_pickup_datetime'))\
                .withColumn('trip_month', month('lpep_pickup_datetime'))

In [0]:
df_trip.display()

In [0]:
df_trip = df_trip.select('VendorID', 'PULocationID', 'DOLocationID', 'fare_amount', 'total_amount')
df_trip.display()

In [0]:
df_trip.write.format('parquet').mode('append').option('path', 'abfss://silver@nycstoragebishwajit.dfs.core.windows.net/trip2023dara').save()

# Analysis

In [0]:
df_trip.display()